In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from ln_utils import *
from transaction_simulator import *

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import sys
from datawand.parametrization import ParamHelper
ph = ParamHelper("../","LNGraph",sys.argv)

# Heatmap plots

In [ ]:
snapshots = range(7)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_7days/"

In [ ]:
amounts = [10000,25000,50000,100000,200000]#,500000]
tx_nums = [1000,2000,5000,10000,20000]

In [ ]:
ts_map = {
    1000:"2019-09-16_20:45:51",
    2000:"2019-09-16_19:13:21",
    5000:"2019-09-16_14:12:54",
    10000:"2019-09-16_17:56:28",
    20000:"2019-09-16_15:39:13"
}

In [ ]:
records = []
for n in tx_nums:
    for a in amounts:
        experiment_id = "%s_%isat_k%i" % (ts_map[n], a, n)
        experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)
        length_distrib = load_data(experiment_folders, snapshots, "lengths_distrib")
        all_length_distrib = pd.concat(length_distrib)
        all_success = all_length_distrib[all_length_distrib["length"]>0]
        avg_path_length = all_success["length"].mean()
        total_counts = all_length_distrib.groupby("length")["count"].sum()
        S = total_counts.sum()
        total_ratios = total_counts / S
        try:
            ratio_failed = total_ratios[-1]
        except KeyError:
            ratio_failed = 0.0
        except:
            raise
        records.append([n,a,ratio_failed,avg_path_length])
        print(n,a)

In [ ]:
ratio_col = "ratio of failed transactions"
avg_path_col = "average path length"
tx_num_col = "number of transactions"

In [ ]:
path_info_df = pd.DataFrame(records, columns=[tx_num_col,"amount",ratio_col,avg_path_col])

In [ ]:
pivot_ratio_failed = path_info_df.pivot(index="amount",columns=tx_num_col,values=ratio_col).sort_values("amount",ascending=False)

In [ ]:
pivot_ratio_avg_length = path_info_df.pivot(index="amount",columns=tx_num_col,values=avg_path_col).sort_values("amount",ascending=False)

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")
cmap = sns.color_palette("coolwarm",12)
fig_s=(6,6)

#### TODO: how many merchants are in the graph in case of 500000SAT?

#### TODO: I should plot the number of transactions with random endpoint?

In [ ]:
fig, ax = plt.subplots(figsize=fig_s)
sns.heatmap(data=pivot_ratio_failed, ax=ax, cmap=cmap)

In [ ]:
fig, ax = plt.subplots(figsize=fig_s)
sns.heatmap(data=pivot_ratio_avg_length, ax=ax, cmap=cmap)

sns.factorplot(data=all_length_distrib, x="snapshot_id", y="ratio", hue="length")

# 1. Load data

In [ ]:
node_names = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_names.csv")

In [ ]:
node_names.head()

In [ ]:
LNBIG_nodes = list(node_names[node_names["is_lnbig"]]["pub_key"])
len(LNBIG_nodes)

In [ ]:
node_names = node_names[["name","pub_key"]]

In [ ]:
def load_data(experiment_files, snapshots, file_postfix):
    parts = []
    for snap_id in snapshots:
        data_files = []
        for f in experiment_files[snap_id]:
            sample_id = f.split(".")[-1]
            df = pd.read_csv("%s/%s.csv" % (f, file_postfix))
            if file_postfix == "lengths_distrib":
                df.columns = ["length","count"]
            df["sample"] = int(sample_id)
            data_files.append(df)
        tmp_df = pd.concat(data_files, axis=0, sort=False)
        tmp_df["snapshot_id"] = snap_id
        parts.append(tmp_df)
        #print(snap_id)
    return parts

#experiment_id = "2019-07-04_11:12:08_200000sat_k30000"
#experiment_id = "2019-07-05_14:39:09_200000sat_k30000"
#experiment_id = "2019-07-05_14:39:09_500000sat_k30000"
#experiment_id = "2019-08-06_13:42:00_200000sat_k30000_aNone"
#experiment_id = "2019-08-09_14:31:08_200000sat_k30000_aNone_dropTrue"
#experiment_id = "2019-08-09_14:31:08_127807sat_k50000_aNone_dropTrue"
#experiment_id = "2019-09-01_18:07:51_200000sat_k30000_aNone_dropTrue"
#experiment_id = "2019-09-02_17:05:18_100000sat_k5000_aNone_dropFalse"
#experiment_id = "2019-09-03_15:28:05_50000sat_k5000_aNone_dropFalse"
#experiment_id = "2019-09-05_14:35:13_50000sat_k6000_aNone_dropTrue"
experiment_id = ph.get("sim_dir")

snapshots = range(7)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations/"

In [ ]:
#experiment_id = ph.get("sim_dir")
experiment_id = "2019-09-06_22:03:19_50000sat_k6000"
snapshots = range(40)#range(54)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_1days/"

In [ ]:
experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)

In [ ]:
experiment_folders[0]

In [ ]:
pricing_pol = load_data(experiment_folders, snapshots, "opt_fees")

In [ ]:
source_fee = load_data(experiment_folders, snapshots, "source_fees")

In [ ]:
router_income = load_data(experiment_folders, snapshots, "router_incomes")

## Router incomes

In [ ]:
router_income_col = "fee"#"income"

In [ ]:
x = snapshots
y = [router_income[i][router_income_col].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
sns.jointplot(data=pd.concat(router_income), x="snapshot_id", y=router_income_col)

In [ ]:
all_router_incomes = pd.concat(router_income)

### LNBIG.com traffic and income timeseries

In [ ]:
lnb_router_incomes = all_router_incomes[all_router_incomes["node"].isin(LNBIG_nodes)].groupby(["node","snapshot_id"]).agg({router_income_col:"mean","num_trans":"mean"}).sort_values(router_income_col,ascending=False).reset_index()

In [ ]:
lnb_router_incomes = lnb_router_incomes.groupby("snapshot_id").agg({router_income_col:"sum","num_trans":"sum"}).reset_index()

In [ ]:
lnb_router_incomes.head()

In [ ]:
sns.lineplot(x="snapshot_id",y="num_trans",data=lnb_router_incomes)

In [ ]:
sns.lineplot(x="snapshot_id",y="fee",data=lnb_router_incomes)

### Calculate average routing income and traffic over snapshots and samples

In [ ]:
all_router_incomes = all_router_incomes.groupby("node").agg({router_income_col:"mean","num_trans":"mean"}).sort_values(router_income_col,ascending=False).reset_index()

In [ ]:
all_router_incomes = all_router_incomes.merge(node_names, left_on="node", right_on="pub_key", how="left").drop("pub_key", axis=1).set_index("node")

In [ ]:
other_routers = [
    "0232e20e7b68b9b673fb25f48322b151a93186bffe4550045040673797ceca43cf",
    "03e50492eab4107a773141bb419e107bda3de3d55652e6e1a41225f06a0bbf2d56",
    "0279c22ed7a068d10dc1a38ae66d2d6461e269226c60258c021b1ddcdfe4b00bc4",
    "03021c5f5f57322740e4ee6936452add19dc7ea7ccf90635f95119ab82a62ae268",
    "03c2abfa93eacec04721c019644584424aab2ba4dff3ac9bdab4e9c97007491dda",
    "03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f",
    "03a503d8e30f2ff407096d235b5db63b4fcf3f89a653acb6f43d3fc492a7674019",
    "03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff402de65c6eb2e1be55dc3e",
    "0331f80652fb840239df8dc99205792bba2e559a05469915804c08420230e23c7c",
    "0217890e3aad8d35bc054f43acc00084b25229ecff0ab68debd82883ad65ee8266",
    "0395033b252c6f40e3756984162d68174e2bd8060a129c0d3462a9370471c6d28f",
    "03cb7983dc247f9f81a0fa2dfa3ce1c255365f7279c8dd143e086ca333df10e278",
    "030c3f19d742ca294a55c00376b3b355c3c90d61c6b6b39554dbc7ac19b141c14f",
    "03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d",
]

In [ ]:
node_income_and_traffic = all_router_incomes.loc[other_routers].reset_index(drop=True)

In [ ]:
node_income_and_traffic

all_router_incomes[(all_router_incomes["num_trans"]>10) & (~all_router_incomes.index.isin(LNBIG_nodes))].head(20)

### LNBIG stats

In [ ]:
lnbig_stats = all_router_incomes.loc[LNBIG_nodes][["fee","num_trans"]].sum()

In [ ]:
lnbig_stats["name"] = "LNBIG.com"

In [ ]:
lnbig_stats

In [ ]:
node_income_and_traffic = node_income_and_traffic.append(lnbig_stats, ignore_index=True).sort_values("name")

In [ ]:
node_income_and_traffic.to_csv("/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/%s.csv" % experiment_id, index=False)

In [ ]:
alma

## Source fees

In [ ]:
x = snapshots
y = [source_fee[i]["mean_fee"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
sns.jointplot(data=pd.concat(source_fee), x="snapshot_id", y="mean_fee")

## Optimal pricing

In [ ]:
x = snapshots
y = [pricing_pol[i]["opt_delta"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
x = snapshots
y = [pricing_pol[i]["opt_traffic"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
x = snapshots
y = [pricing_pol[i]["origi_income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
x = snapshots
y = [pricing_pol[i]["opt_income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

### Calculate income revenue

In [ ]:
for df in pricing_pol:
    df["income_diff"] = df["opt_income"] - df["origi_income"]

# 2. Simulation stability

In [ ]:
corrs = ["spearman","kendall","wkendall"]

In [ ]:
def avg_cross_corr(df, snapshot_id, col, methods=corrs, key_col="node"):
    snap = df[snapshot_id]
    sample_num = snap["sample"].max()+1
    cols = [key_col, col]
    merged = snap[snap["sample"]==0][cols].rename({col:col+"_0"}, axis=1)
    for i in range(1,sample_num):
        s = snap[snap["sample"]==i][cols].rename({col:col+"_%i" % i}, axis=1)
        merged = merged.merge(s, on=key_col, how="outer").fillna(0.0)
    merged.drop("node", inplace=True, axis=1)
    res = {}
    for method in methods:
        cnt = corr_mx(merged, method=method).sum().sum() - sample_num 
        denom = sample_num**2-sample_num
        res[method] = cnt / denom
    return res

In [ ]:
origi_inc = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "origi_income") for snap_id in snapshots])
opt_inc = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "opt_income") for snap_id in snapshots])
income_diff = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "income_diff") for snap_id in snapshots])
opt_delta = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "opt_delta") for snap_id in snapshots])

In [ ]:
colors = ['b','g','r','m']
fig, axis = plt.subplots(1,3,figsize=(15,4))
x = snapshots
# incomes
axis[0].set_title("INCOME mean cross correlations")
for i, c in enumerate(corrs):
    axis[0].plot(x,origi_inc[c], colors[i]+'-', label=c+"_orig")
    axis[0].plot(x,opt_inc[c], colors[i]+'--', label=c+"_opt")
axis[0].set_xlabel("snapshot")
axis[0].legend()
# income_diff
axis[1].set_title("INCOME_DIFF mean cross correlations")
for i, c in enumerate(corrs):
    axis[1].plot(x,income_diff[c], colors[i]+'-', label=c)
axis[1].set_xlabel("snapshot")
axis[1].legend()
# opt_delta
axis[2].set_title("OPT_DELTA mean cross correlations")
for i, c in enumerate(corrs):
    axis[2].plot(x,opt_delta[c], colors[i]+'-', label=c)
axis[2].set_xlabel("snapshot")
axis[2].legend()
plt.show()

#### Incomes observations

- taking mean cross correlation of user incomes (original and optimal)
- weighted kendall-tau gives decreased to 0.90 from 0.95 (after including capacity maintenance)
- in case of unweighted spearman and kendall the optimal income correlates better across samples 

#### income_diff observations¶

- taking mean cross correlation of income difference (optimal income - original income)
- weighted kendall-tau decreased to 0.80 from 0.93 (after including capacity maintenance)

#### opt_delta observations¶

- taking mean cross correlation of optimal base fee change
- weighted kendall-tau decreased to 0.70 from 0.90 (after including capacity maintenance)

**CONCLUSION: our base_fee optimization procedure is less efficient (meaningful) after the implementation of capacity maintenance!!!**

# 3. Income revenue analysis

In [ ]:
def cut_into_categories(df):
    #categories = [df[:10], df[10:30], df[30:50], df[50:100]]
    #categories = [df[:20], df[20:50], df[50:100], df[100:200]]
    #categories = [df[:10], df[10:20], df[20:30], df[30:40], df[40:50], df[50:100], df[100:500]]
    categories = [df[:10], df[10:20], df[20:50], df[50:100], df[100:500]]
    #categories = [df[:50], df[50:100], df[100:500]]#, df[200:500]]
    return categories

def merge_categories(categories):
    for i, cat in enumerate(categories):
        cat["category"] = i
    return pd.concat(categories)
    
def show_stats(snap_id, col="income_diff"):
    mean = pricing_pol[snap_id].groupby("node").mean().sort_values("origi_income", ascending=False)
    categories = cut_into_categories(mean)
    return [float(cat[[col]].mean()) for cat in categories]

In [ ]:
x = range(5)
#x = range(7)

### Visualize each snapshot

df = pd.DataFrame([show_stats(i, "income_diff") for i in snapshots])

plt.Figure(figsize=(10,10))
for i in snapshots:
    plt.plot(x, show_stats(i, "income_diff"), label=i)
plt.yscale("log")
plt.legend()

plt.Figure(figsize=(10,10))
for i in snapshots:
    plt.plot(x, show_stats(i, "opt_traffic"), label=i)
plt.legend()

### Average stats (over all snapshots and samples)

average_stats = pricing_pol[0].groupby("node").mean().sort_values("origi_income", ascending=False)[["income_diff","origi_income","opt_income","opt_traffic","opt_delta"]]

In [ ]:
all_records = pd.concat([pricing_pol[i] for i in snapshots])

In [ ]:
average_stats = all_records.groupby("node").mean().sort_values("origi_income", ascending=False)[["income_diff","origi_income","opt_income","opt_traffic","opt_delta"]]

In [ ]:
average_stats_cats = cut_into_categories(average_stats)
mean_diff = [float(cat[["income_diff"]].mean()) for cat in average_stats_cats]
mean_traffic = [float(cat[["opt_traffic"]].mean()) for cat in average_stats_cats]
mean_delta = [float(cat[["opt_delta"]].mean()) for cat in average_stats_cats]

In [ ]:
fig, axis = plt.subplots(1,3,figsize=(15,4))
axis[0].plot(x, mean_diff, 'rx')
axis[0].set_xlabel("income category")
axis[0].set_ylabel("mean income_diff")
axis[1].plot(x, mean_delta, 'g*')
axis[1].set_xlabel("income category")
axis[1].set_ylabel("mean opt_delta")
axis[2].plot(x, mean_traffic, 'bo')
axis[2].set_xlabel("income category")
axis[2].set_ylabel("mean opt_traffic")
plt.show()

#### Mean original income for the selected categories

In [ ]:
[float(cat[["origi_income"]].mean()) for cat in average_stats_cats]

### PROBLEM with outliers...

In [ ]:
merged = merge_categories(average_stats_cats)

In [ ]:
sns.boxplot(data=merged, x="category", y="income_diff", whis=2)

average_stats.head(10)

### Nodes above 20K SAT (2.5 USD) income_diff (with alpha=2.0 source distribution)

#### 2019-07-04_11:12:08_200000sat_k30000 (ordered by original income)

1. yalls.org - Read and write articles, with Lightning Network micropayments. (195,925.46 USD)
2. ACINQ - ACINQ is one of the leading companies working on Bitcoin scalability. (507,211.38 USD)
3. 1ML.com node ALPHA (89,453.40 USD)
4. LightningPowerUsers.com (313,406.05 USD)
5. LightningTo.Me - Helping to resolve routing and capacity issues (165,531.35 USD)
6. tomjodh.nl - Buy photos with satoshis (8,709.95 USD) - **TODO RECLASSIFY AS merchant!!!**
7. ? - ? (848.05 USD)

#### 2019-07-05_14:39:09_200000sat_k30000 (ordered by original income)

1. ACINQ - ACINQ is one of the leading companies working on Bitcoin scalability. (507,211.38 USD, 783 channel)
2. 1ML.com node ALPHA (89,453.40 USD, 884 channel)
3. LightningPowerUsers.com (313,406.05 USD, 1,266 channel)
4. LightningTo.Me - Helping to resolve routing and capacity issues (165,531.35 USD, 700 channel)
5. tomjodh.nl - Buy photos with satoshis (8,709.95 USD, 47 channel)
6. ? - ? (859.23 USD, 4 channel)
7. nybemotion - ? (318.35 USD, 3 channel)

#### Additional nodes in 2019-07-05_14:39:09_500000sat_k30000

4. Bitrefill.com - Buy Gift Cards with Bitcoin for 30+ supported services (310,369.63 USD, 305 channel) - **TODO RECLASSIFY AS merchant!!!**
6. tady je slushovo: ? (48,872.23 USD, 336 channel)

### Nodes above 20K SAT (2.5 USD) income_diff (with alpha=None source distribution)

#### 2019-08-06_13:42:00_200000sat_k30000_aNone (ordered by original income)

1. yalls.org
2. ACINQ
3. LightningPowerUsers.com
4. tomjodh.nl
5. Node: 0297b368360113e29032 ???
6. nybemotion

#### 2019-08-09_14:31:08_200000sat_k30000_aNone_dropTrue

1. yalls.org
2. ln1.satoshilabs.com (306,117.70 USD - 824 channels) **(NEW)**
3. LightningPowerUsers.com
4. nybemotion (493.61 USD, 6 channel) ???
5. Amazing_Droopy_3 (113.73 USD, 1 channel?) ???

#### 2019-09-01_18:07:51_200000sat_k30000_aNone_dropTrue (WITH capacity maintenance)

1. LightningPowerUsers.com (237,653 USD, 1261 channels)
2. 1ML.com node ALPHA (64,630 USD, 889 channels)
3. CoinGate (147,272 USD, 462 channels) - CoinGate is the first large cryptocurrency payment gateway to introduce Lightning Network payments to hundreds of merchants with no technical knowledge required
4. lndhub.ru (19,066 USD, 248 channels) - increase connectivity
5. BeerselLightningNode (13,246 USD, 84 channels)
6. BitKassa LN34961 (13,631 USD, 47) - some kind of payment API and BTC tech company
7. Serving Beer (2,107 USD, 13 channels) - is this really a merchant???

In [ ]:
average_stats[average_stats["income_diff"]>20000]